ThreeVariableLP-Model3 and TwoVariableLP-Model3 are similar.
We have the same model - model3.mod
The only difference is in model3a.dat and model3b.dat. 
In model3b.dat, we added an extra variable.

In [1]:
from amplpy import AMPL, Environment
#this is for windows only. after dockerizing not required anymore.
#ampl = AMPL(Environment('D:\\amplide.mswin64\\amplide.mswin64')) 
ampl = AMPL(Environment('/opt/ampl.linux64')) 

In [2]:
#ampl.option['solver'] = 'cplexamp'
#we can do this to set the solver we want.

In [3]:
ampl.read('./Model3.mod')

In [4]:
ampl.readData('./Model3b.dat')

In [5]:
print(ampl.getObjective('Total_Profit').get())
print(ampl.getConstraint('Time').get())

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];
subject to Time:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;


In [6]:
ampl.eval('expand Total_Profit;')
ampl.eval('expand Time;')

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];

subject to Time:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;



In [7]:
ampl.solve()

MINOS 5.51: optimal solution found.
2 iterations, objective 196400


In [8]:
print(ampl.getObjective('Total_Profit').get().value())

196400.0


##### Note that the profits have increased between TwoVariableLP-Model3.ipynb and ThreeVariableLP-Model3.ipynb by adding the new variable "plate".

In [9]:
ampl.eval('expand X["bands"];')
ampl.eval('expand X["coils"];')
ampl.eval('expand X["plate"];')

Coefficients of X['bands']:
	Time           0.005
	Total_Profit  25

Coefficients of X['coils']:
	Time           0.00714286
	Total_Profit  30

Coefficients of X['plate']:
	Time           0.00625
	Total_Profit  29



In [10]:
ampl.eval('display _nvars, _ncons;')
#displays number of variables and number of constraints

_nvars = 3
_ncons = 1



#### If the system/problem number of constraints is much larger than number of variables; this will result in: "Problem has too few degrees of freedom".  There might be redundancy in these constraint. Use the command expand <Constraint_Name>; and check the constraints generated by AMPL. If "\_ncons" >> "\_nvars" then we may have misunderstood the requirements and unnecessarily defined constraints in the model.


In [11]:
print(ampl.getVariable('X').get('bands'))
print(ampl.getVariable('X').get('coils'))
print(ampl.getVariable('X').get('plate'))
print(ampl.getVariable('X').get('bands').value())
print(ampl.getVariable('X').get('coils').value())
print(ampl.getVariable('X').get('plate').value())

var X['bands'] >=0, <=6000;
var X['coils'] >=0, <=4000;
var X['plate'] >=0, <=3500;
6000.0
0.0
1599.9999999999998


In [12]:
print(ampl.getData('X'))

   index0    |      X      
  'bands'    |     6000    
  'coils'    |      0      
  'plate'    | 1599.9999999999998



In [13]:
values = ampl.getVariable('X').getValues()
print(values)

   index0    |    X.val    
  'bands'    |     6000    
  'coils'    |      0      
  'plate'    | 1599.9999999999998



In [14]:
df = values.toPandas()
print(df)

        X.val
bands  6000.0
coils     0.0
plate  1600.0


In [15]:
l = values.toList()
print(l)

[('bands', 6000.0), ('coils', 0.0), ('plate', 1599.9999999999998)]


In [16]:
d = values.toDict()
print(d)

{'bands': 6000.0, 'coils': 0.0, 'plate': 1599.9999999999998}


| TwoVariableLP                    | ThreeVariableLP           |
| :---                             |  ---:                     |
| Bands = 6000.0                   | Bands = 6000.0            |
| Coils = 1399.9999999999993       | Coils =    0.0            |
| -                                | Plate = 1599.9999999999993|

In the ThreeVariableLP case, Coils went to 0.0 while Plate increased to 1599.99.
This makes sense because, profit per hour for :
*  bands = 200 tons/hour * 25 dollar/ton = 5000 dollar/hour
*  coils = 140 tons/hour * 30 dollar/ton = 4200 dollar/hour
*  plate = 160 tons/hour * 29 dollar/ton = 4640 dollar/hour

Based on the hourly profits, bands are the most profitable. So we make as much bands as possible but we are limited to make 6000 tons of bands. The next most profitable is plate, whatever remaining hours are available in the week, we make plate till we reach the maximum limit of 3500 tons of plate.